In [ ]:
import cv2
import dlib
import numpy as np
import os
import math

# Load detector and predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(
    "models/shape_predictor_68_face_landmarks.dat/shape_predictor_68_face_landmarks.dat"
)

# Define the root directory for your emotion folders
root_input_dir = "archive_3"
root_output_dir = "processed_faces_new"

# Define the scaling factor for upscaling images
scale_factor = 6

# Create the root output directory if it doesn't exist
os.makedirs(root_output_dir, exist_ok=True)

# List of emotion folders (assuming these are the names of your subfolders)
emotion_folders = ["anger", "contempt", "disgust", "happy", "sadness", "fear", "surprise"]

for emotion in emotion_folders:
    input_emotion_path = os.path.join(root_input_dir, emotion)
    output_emotion_path = os.path.join(root_output_dir, emotion)

    # Create output directory for the current emotion if it doesn't exist
    os.makedirs(output_emotion_path, exist_ok=True)
    print(f"Processing emotion: {emotion}")

    # Iterate through each image in the emotion folder
    for filename in os.listdir(input_emotion_path):
        if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
            image_path = os.path.join(input_emotion_path, filename)
            image = cv2.imread(image_path)

            if image is None:
                print(f"Warning: Could not read image {image_path}. Skipping.")
                continue

            # Upscale the image
            upscaled_image = cv2.resize(image, (0, 0), fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)
            gray = cv2.cvtColor(upscaled_image, cv2.COLOR_BGR2GRAY)

            # Detect faces
            faces = detector(gray)

            if len(faces) > 0:
                # Process the first detected face (you can modify this if you need to process all faces)
                face = faces[0]
                landmarks = predictor(gray, face)

                drawn_image = upscaled_image.copy() # Draw landmarks on a copy of the upscaled image

                # Get position of landmark 31 (index 30 in 0-based indexing)
                nose_x = landmarks.part(30).x
                nose_y = landmarks.part(30).y

                # Draw quadrant lines centered at landmark 31
                img_h, img_w = drawn_image.shape[:2]
                cv2.line(drawn_image, (nose_x, 0), (nose_x, img_h), (0, 255, 0), 1)
                cv2.line(drawn_image, (0, nose_y), (img_w, nose_y), (0, 255, 0), 1)

                drawn_index = 1
                seen = set()

                for n in range(68):
                    x = landmarks.part(n).x
                    y = landmarks.part(n).y

                    if (x, y) in seen:
                        continue
                    seen.add((x, y))

                    # Special highlight for landmark 31
                    if n == 30:
                        cv2.circle(drawn_image, (x, y), 5, (0, 0, 255), -1)
                        cv2.putText(drawn_image, str(drawn_index), (x - 12, y - 12),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2, cv2.LINE_AA)
                    else:
                        # Draw standard landmark
                        cv2.circle(drawn_image, (x, y), 2, (0, 0, 255), -1)

                        # Offset for label
                        offset_x, offset_y = -6, 10

                        if n in [37, 38, 39, 43, 44, 45]:
                            offset_y = -10
                        elif n == 49:
                            offset_x = -20
                            offset_y = 6
                        elif n == 50:
                            offset_x = 0
                            offset_y = -8
                        elif n == 65:
                            offset_x = -14
                            offset_y = 10
                        elif 48 <= n <= 54:
                            offset_y = -8
                        elif 55 <= n <= 59:
                            offset_y = 10
                        elif 60 <= n <= 64:
                            offset_y = -8
                        elif 65 <= n <= 67:
                            offset_y = 10

                        # Draw label
                        cv2.putText(drawn_image, str(drawn_index), (x + offset_x, y + offset_y),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 0), 1, cv2.LINE_AA)

                    drawn_index += 1

                # Save the processed image
                output_image_path = os.path.join(output_emotion_path, filename)
                cv2.imwrite(output_image_path, drawn_image)
                # print(f"Saved processed image: {output_image_path}")

            else:
                print(f"No face detected in {filename} from {emotion} folder.")

print("\nAll emotion folders processed.")

Processing emotion: anger
Processing emotion: contempt
Processing emotion: disgust
Processing emotion: happy
Processing emotion: sadness
Processing emotion: fear
Processing emotion: surprise

All emotion folders processed.
